僅須執行第一次

In [ ]:
%cd "/content/drive/MyDrive/Colab Notebooks"

/content/drive/MyDrive/Colab Notebooks


In [ ]:
!pip install gradio
!pip install tensorflow
!pip install numpy
!pip install Pillow

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 57.2/57.2 MB 12.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 320.2/320.2 kB 18.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.8/94.8 kB 8.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 11.2/11.2 MB 75.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 73.2/73.2 kB 6.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 63.8/63.8 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.2/168.2 kB 14.2 MB/s eta 0:00:00
  Attempting uninstall: markupsafe
    Found existing installation: MarkupSafe 3.0.2
    Uninstalling MarkupSafe-3.0.2:
      Successfully uninstalled MarkupSafe-3.0.2


In [ ]:

# 指定 .zip 檔案的路徑（請替換為您雲端硬碟的實際路徑）
zip_path = '/content/drive/MyDrive/archive.zip'  # 替換為您的檔案路徑
extract_path = '/content/drive/MyDrive/Colab Notebooks/dataset'  # 解壓縮目標資料夾

# 解壓 .zip 檔案
with zipfile.ZipFile(zip_path, 'r') as zip_ref:
    zip_ref.extractall(extract_path)
print(f"資料已解壓縮到 {extract_path}")

資料已解壓縮到 /content/drive/MyDrive/Colab Notebooks/dataset


每次都要執行

In [ ]:
from google.colab import drive
import zipfile
import os

In [ ]:
# 假設資料已解壓縮到 /content/archive/
import tensorflow as tf
from tensorflow.keras import layers, models
from tensorflow.keras.applications import ResNet50
import numpy as np
import os
# 匯入回調函數模組
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau

# Step 1: 定義資料路徑
train_path = "/content/drive/MyDrive/Colab Notebooks/dataset/flags/train"  # 訓練集路徑
test_path = "/content/drive/MyDrive/Colab Notebooks/dataset/flags/test"   # 測試集路徑
image_size = (224, 224)
batch_size = 32

# Step 2: 載入數據集
train_dataset = tf.keras.utils.image_dataset_from_directory(
    train_path,
    image_size=image_size,
    batch_size=batch_size
)

test_dataset = tf.keras.utils.image_dataset_from_directory(
    test_path,
    image_size=image_size,
    batch_size=batch_size
)
class_names = train_dataset.class_names
# 數據增強與預處理
data_augmentation = tf.keras.Sequential([
    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.2),
])

def preprocess(image, label):
    image = data_augmentation(image)
    image = tf.cast(image, tf.float32) / 255.0  # 正規化到 [0, 1]
    return image, label

train_dataset = train_dataset.map(preprocess)
test_dataset = test_dataset.map(preprocess)

# Step 3: 模型構建
def create_model(num_classes):
    base_model = ResNet50(weights="imagenet", include_top=False, input_shape=(224, 224, 3))
    base_model.trainable = False  # 冻結預訓練權重

    model = models.Sequential([
        base_model,
        layers.GlobalAveragePooling2D(),
        layers.Dense(256, activation="relu"),
        layers.Dropout(0.5),
        layers.Dense(num_classes, activation="softmax")  # num_classes 是國旗分類數
    ])
    return model

num_classes = len(class_names)  # 根據 train 資料夾中的子資料夾數量確定類別數
model = create_model(num_classes)

# 編譯模型
model.compile(optimizer="adam",
              loss="sparse_categorical_crossentropy",
              metrics=["accuracy"])


Found 19716 files belonging to 249 classes.
Found 5154 files belonging to 249 classes.
94765736/94765736 ━━━━━━━━━━━━━━━━━━━━ 5s 0us/step


In [ ]:
# Step 4: 模型訓練
print("開始訓練模型...")
history = model.fit(
    train_dataset,
    validation_data=test_dataset,
    epochs=10
)

# Step 5: 保存模型
model.save("resnet_flag_classifier.h5")
print("模型已保存為 resnet_flag_classifier.h5")

開始訓練模型...
Epoch 1/10
617/617 ━━━━━━━━━━━━━━━━━━━━ 342s 528ms/step - accuracy: 0.0056 - loss: 5.5120 - val_accuracy: 0.0116 - val_loss: 5.2784
Epoch 2/10
617/617 ━━━━━━━━━━━━━━━━━━━━ 307s 498ms/step - accuracy: 0.0125 - loss: 5.2990 - val_accuracy: 0.0402 - val_loss: 5.0707
Epoch 3/10
617/617 ━━━━━━━━━━━━━━━━━━━━ 317s 489ms/step - accuracy: 0.0225 - loss: 5.1172 - val_accuracy: 0.0601 - val_loss: 4.8721
Epoch 4/10
617/617 ━━━━━━━━━━━━━━━━━━━━ 316s 510ms/step - accuracy: 0.0286 - loss: 4.9877 - val_accuracy: 0.0753 - val_loss: 4.7349
Epoch 5/10
617/617 ━━━━━━━━━━━━━━━━━━━━ 318s 515ms/step - accuracy: 0.0338 - loss: 4.8811 - val_accuracy: 0.0825 - val_loss: 4.6351
Epoch 6/10
617/617 ━━━━━━━━━━━━━━━━━━━━ 290s 469ms/step - accuracy: 0.0438 - loss: 4.8259 - val_accuracy: 0.0986 - val_loss: 4.5408
Epoch 7/10
617/617 ━━━━━━━━━━━━━━━━━━━━ 278s 449ms/step - accuracy: 0.0461 - loss: 4.7631 - val_accuracy: 0.1003 - val_loss: 4.4536
Epoch 8/10
617/617 ━━━━━━━━━━━━━━━━━━━━ 311s 503ms/step - accuracy

模型已保存為 resnet_flag_classifier.h5


In [ ]:
# Step 6: 使用 Gradio 建立使用者介面
import gradio as gr
from PIL import Image

# **Use the stored class_names variable instead of trying to access it from train_dataset**
# class_names = list(train_dataset.class_names)  # This line is causing the error
# class_names is already available from previous cell execution

# 定義預測函數
def predict_flag(image):
    # 預處理圖片
    image = image.resize((224, 224))
    image = np.array(image) / 255.0
    image = np.expand_dims(image, axis=0)

    # 模型預測
    predictions = model.predict(image)
    predicted_class = class_names[np.argmax(predictions)]
    confidence = np.max(predictions)

    return f"國旗: {predicted_class}, 置信度: {confidence:.2f}"



In [ ]:
# 建立 Gradio 介面
interface = gr.Interface(
    fn=predict_flag,
    inputs=gr.Image(type="pil", label="上傳國旗圖片"),
    outputs="text",
    title="國旗分類 AI",
    description="上傳國旗圖片，AI 將自動辨識國家的名稱。"
)

# 啟動介面
interface.launch()

Running Gradio in a Colab notebook requires sharing enabled. Automatically setting `share=True` (you can turn this off by setting `share=False` in `launch()` explicitly).

Colab notebook detected. To show errors in colab notebook, set debug=True in launch()
* Running on public URL: https://edaf21ddb585715c73.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from the terminal in the working directory to deploy to Hugging Face Spaces (https://huggingface.co/spaces)
